In [1]:
# 載入模組
from urllib.request import Request, urlopen
from datetime import datetime, timedelta, timezone
from dateutil import tz
from IPython.display import display
import time
import json
import math
import pandas as pd
import numpy as np

# 查詢幣種API
pair_api = ''

# 查詢幣種資料API
kline_api = ''

# 想查詢幣種，可以參個下列格式自行修改（請先確認想查詢的幣種有在幣安交易所，目前查詢幣對只支援USDT）
interested_list = ['AAVE', 'BTC', 'WAN', 'WAVES', 'WRX', 'XEM', 'XRP', 'XTZ', 'XVS', 'XVG', 'YFI', 'ZIL']

# 查詢起始日（含當天），欄位依序為：年、月、日
_start_date = [2021, 5, 19]

# 查詢結束日（含當天），欄位依序為：年、月、日
_end_date = [2021, 5, 31]

# 如果查詢太多幣種，查詢日期範圍也太廣，建議開啟延遲delay_set這個選項（開啟請把0改為1）；delay_time為延遲時間（單位為秒）
delay_set = 0
delay_time = 1

def get_data(pair_api=None, kline_api=None, v=None, mode=None):
  url = ''
  headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

  if v is not None:
    info = []
    df = []
    startTime = int(time.mktime(datetime.strptime(datetime(v[1][0][0], v[1][0][1], v[1][0][2], 0, 0, tzinfo=tzinfo).astimezone().isoformat(), '%Y-%m-%dT%H:%M:%S+00:00').timetuple()))
    endTime = int(time.mktime(datetime.strptime(datetime(v[1][1][0], v[1][1][1], v[1][1][2], 0, 0, tzinfo=tzinfo).astimezone().isoformat(), '%Y-%m-%dT%H:%M:%S+00:00').timetuple()))

    diffTime = (endTime - startTime) + (24 * 60 * 60)
    loop_floor = math.floor(diffTime / 60 / 1000)
    loop_ceil = math.ceil(diffTime / 60 / 1000)
  else:
    url = pair_api
    r = Request(url, headers=headers)
    data = json.loads(urlopen(r).read())

    d = {}

    for i in range(len(data['data'])):
      if data['data'][i]['quote'] == 'USDT':
        d.setdefault(data['data'][i]['base'], [data['data'][i]['symbol'], (_start_date, _end_date)])

  if mode == '1d':
    url = f'{kline_api}?symbol={v1[0]}&interval=1d&limit=1000&startTime={startTime}000&endTime={endTime}000'
    r = Request(url, headers=headers)
    data = json.loads(urlopen(r).read())

    if len(data) > 0:
      info.append(data)
  
  if mode == '1m':
    delta = 0
    
    for i in range(loop_ceil):
      if (i + 1) > loop_floor:
        delta = endTime - (startTime + ((i + 1) * 60 * 1000)) + (24 * 60 * 60)
      url = f'{kline_api}?symbol={v1[0]}&interval=1m&limit=1000&startTime={startTime + (i * 60 * 1000)}000&endTime={startTime + ((i + 1) * 60 * 1000) + delta}000'     
      r = Request(url, headers=headers)
      data = json.loads(urlopen(r).read())

      if len(data) > 0:
        info.append(data)

  if v is not None:
    if len(info) > 0:
      a = np.concatenate(info)
      data = None
      df = pd.DataFrame(a)

      df[0] = pd.to_datetime((pd.to_numeric(df[0]).astype('Int64') / 1000), unit='s').dt.tz_localize('UTC').dt.tz_convert(tz_convert).dt.tz_localize(None)
      df[6] = pd.to_datetime((pd.to_numeric(df[6]).astype('Int64')), unit='ms').dt.tz_localize('UTC').dt.tz_convert(tz_convert).dt.tz_localize(None)

    return df
  else:
    return d

def daily_status(daily):
  _change = 0

  if float(daily[1]) < float(daily[4]):
    _change = (float(daily[2]) / float(daily[3])) - 1
  elif float(daily[1]) > float(daily[4]):
    _change = (1 - (float(daily[3]) / float(daily[2]))) * -1
  
  return '{:.2%}'.format(_change)

_search = get_data(pair_api, kline_api)
tz_convert = None
zone = tz.gettz('UTC')
tzinfo = zone
symbol = []
df_data_all = []

for k1, v1 in _search.items():
  if k1 not in interested_list:
    continue

  symbol.append(k1)
  df_1d = get_data(pair_api, kline_api,v1, '1d')

  if len(df_1d) < 1:
    continue

  if datetime.strftime(df_1d.loc[0, 0], '%Y-%m-%d') != datetime.strftime(datetime(v1[1][0][0], v1[1][0][1], v1[1][0][2], 0, 0, tzinfo=tzinfo), '%Y-%m-%d'):
    continue

  df_1m = get_data(pair_api, kline_api, v1, '1m')

  symbol_data = []
  df_data = []

  for j in df_1d.loc[:,:4].values:
    df_1m_subset = df_1m[df_1m[0].dt.strftime('%Y-%m-%d').isin([datetime.strftime(j[0], '%Y-%m-%d')])].reset_index(drop=True)        
    array_1m_high = df_1m_subset.loc[np.nonzero(np.in1d(pd.to_numeric(df_1m_subset[2]).astype(float), float(j[2])))[0][0]].tolist()
    array_1m_low = df_1m_subset.loc[np.nonzero(np.in1d(pd.to_numeric(df_1m_subset[3]).astype(float), float(j[3])))[0][0]].tolist()
    symbol_data.append([array_1m_high[0], array_1m_high[2], array_1m_low[0], array_1m_low[3], daily_status(j)])
    
  df_data = pd.DataFrame(symbol_data, columns=pd.MultiIndex.from_arrays([['高點', '高點', '低點', '低點', '漲跌'], ['時間', '價格', '時間', '價格', '']])).T
  df_data.columns = pd.to_datetime(df_data.iloc[0]).dt.strftime('%Y-%m-%d').tolist()
  df_data = df_data.T
  df_data[('高點', '價格')] = pd.to_numeric(df_data[('高點', '價格')]).astype(float).round(decimals=4).astype(str)
  df_data[('低點', '價格')] = pd.to_numeric(df_data[('低點', '價格')]).astype(float).round(decimals=4).astype(str)
  df_data[('高點', '時間')] = pd.to_datetime(df_data[('高點', '時間')]).dt.strftime('%H:%M').tolist()
  df_data[('低點', '時間')] = pd.to_datetime(df_data[('低點', '時間')]).dt.strftime('%H:%M').tolist()
  df_data_all.append(df_data.T.to_dict())

  if delay_set:
    time.sleep(delay_time)

df_result = pd.concat([pd.DataFrame(symbol, columns=pd.MultiIndex.from_arrays([[''], [''], ['幣種']])), pd.concat([pd.DataFrame(pd.DataFrame.from_dict(x).unstack([0, 1])).T for x in df_data_all]).reset_index(drop=True).T.reindex(['高點', '低點', '漲跌'], level=1).T], axis=1)
df_result2 = df_result.copy()
df_result2 = df_result2.reindex(list(df_result[df_result[('', '', '幣種')] == 'BTC'].index) + [idx for idx in range(len(df_result.iloc[:, 0])) if idx != df_result[df_result[('', '', '幣種')] == 'BTC'].index])
display(df_result2)

2021-05-19                  ... 2021-05-31                         
                  高點              低點  ...         高點     低點                漲跌
       幣種         時間       價格     時間  ...         價格     時間        價格        
1     BTC      00:14  43584.9  13:10  ...    37499.0  04:30  34153.84   9.79%
0    AAVE      00:29   649.86  12:56  ...     385.41  04:30    304.36  26.63%
2     WAN      00:15   1.4916  13:10  ...     0.9375  04:24    0.8102  15.71%
3   WAVES      00:27    28.02  13:08  ...     13.982  03:05    12.124  15.32%
4     WRX      00:14   2.0053  13:10  ...     2.3582  03:10    1.2601  87.14%
5     XEM      00:16    0.292  13:10  ...      0.198  04:23    0.1721  15.05%
6     XRP      00:27   1.6448  12:55  ...     1.0495  03:10    0.8789  19.41%
7     XTZ      00:11   5.8377  13:11  ...     3.6127  03:10     3.144  14.91%
8     XVS      00:08   83.936  13:10  ...     33.556  04:30    29.184  14.98%
9     YFI      00:11  74565.2  13:10  ...    47280.0  04:23   40000.0  18.20%
10    ZIL      00:14   0.1778  13:09  ...      0.116  03:10    0.1005  15.45%

[11 rows x 66 columns]